## Imports

In [1]:
from dataclasses import dataclass, field
import torch
import random
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
from transformers import RobertaTokenizer, RobertaForMaskedLM
from typing import List
from torch.utils.data import DataLoader, Dataset, Subset
from model import RobertaClassificationAndLM
from data import EthicsDataset, MoralStoriesDataset, WikiTextDataset, morality_classification_examples, morality_probing_examples_hard
from datasets import load_dataset
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
from pynvml import *
from matplotlib.pyplot import figure
import time

from helper import create_attention_mask, calculate_accuracy_loss, train_model, print_gpu_utilization, get_gpu_mem_usage, calculate_wikitext_loss


if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
    
print(f"using device: {device}")

torch.set_float32_matmul_precision('high')

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

@dataclass 
class RobertaBaseConfig:
    mod_layers: list = field(default_factory=lambda: list(range(12)))
    vocab_size: int = 50265
    hidden_size: int = 768
    num_hidden_layers: int = 12
    num_attention_heads: int = 12
    intermediate_size: int = 3072
    max_position_embeddings: int = 514
    layer_norm_eps: float = 1e-05
    num_class_labels: int = 1
    pad_token_id: int = 1

    # Special Configs 
    rank: int = None
    attn_type: str = 'spda'
    use_bottleneck: bool = False
    bottleneck_size: int = None
    prefix_size: int = None
    use_prefix: bool = False


c:\Users\zfogg\anaconda3\envs\torch-cuda12.4\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using device: cuda


In [2]:
d = torch.load('./Datasets/ethics/test_dataset_ethics.pt', weights_only= False)
d = DataLoader(d, batch_size=32)
m = RobertaClassificationAndLM.from_pretrained(RobertaBaseConfig(use_bottleneck = True, bottleneck_size = 8, mod_layers= list(range(6,12))), size = 'base')
m.load_state_dict(torch.load('./trained_models/base/adapter_900', weights_only= True))

m = m.to(device)

print(calculate_accuracy_loss(m, d, device, 0))

(49.267169179229484, 49.0787269681742, 0.08937899768352509, 1.5672330856323242)


## Create Models

In [7]:
moral_pref = "This is moral: "
immoral_pref = "This is immmoral: "
neutral_pref  = "This is neutral: "

def get_probs(model, x, device, prefix_size):
    x = tokenizer.encode(x)
    x = torch.tensor(x).unsqueeze(0).to(device)
    
    attn_mask = create_attention_mask(x, device, dtype = torch.bfloat16, prefix_size= prefix_size)
    attn_mask = attn_mask.to(torch.float32)

    with torch.no_grad() and torch.autocast(device_type = device, dtype = torch.bfloat16):
        logits, _, _ = model(x, attention_mask = attn_mask, run_lm_head = True)

    mask_token_index = (x == tokenizer.mask_token_id)[0].nonzero(as_tuple = True)[0]

    probs = F.softmax(logits[0, mask_token_index], dim=1)

    return probs 

def collect_ratios(model, data, device, tokenizer, prefix_size = 0):
    seq = data["Seq"]
    moral_token = tokenizer.encode(data["Moral"])[1:-1]
    immoral_token = tokenizer.encode(data["Immoral"])[1:-1]

    if len(moral_token) != 1 or len(immoral_token) != 1: 
        print(f'Could not encoder targets to single token: {data}' )
        return None, None, None
    
    moral_token = moral_token[0]
    immoral_token = immoral_token[0]

    moral_probs = get_probs(model, moral_pref + seq, device, prefix_size= prefix_size).view(-1)
    immoral_probs = get_probs(model, immoral_pref + seq, device, prefix_size= prefix_size).view(-1)
    neutral_probs = get_probs(model, neutral_pref + seq, device, prefix_size= prefix_size).view(-1)

    # How much more likely model thinks that moral token is compared to the immoral token

    # Ratio for model that was hopefully conditioned to choose a produce moral text
    moral_ratio = (moral_probs[moral_token] / moral_probs[immoral_token]).item() 
    
    # Ratio for model that was hopefully conditioned to choose a produce immoral text
    immoral_ratio = (immoral_probs[moral_token] / immoral_probs[immoral_token]).item()

    # Ratio for model that was not conditioned to produce moral or immoral text
    neutral_ratio = (neutral_probs[moral_token] / neutral_probs[immoral_token]).item()

    return moral_ratio, neutral_ratio, immoral_ratio

def get_top_k_preds(model, x, device, prefix_size = 0, k = 5):
    probs = get_probs(model, x, device, prefix_size)
    
    topk = torch.topk(probs, k)

    topk = [(tokenizer.decode(topk.indices.squeeze()[i].item()), round(topk.values.squeeze()[i].item() * 100, 2)) for i in range(topk.indices.shape[1])]
    
    return topk


def moral_prediction_accuracy(model, dataset, device, tokenizer, prefix_size = 0):

    correct = 0

    for data in dataset:
        moral_ratio, neutral_ratio, immoral_ratio = collect_ratios(model, data, device, tokenizer, prefix_size)

        '''
        If model has learned to output moral/immmoral text, then it moral_ratio should be higher then neutral since the model should     
        prefer the moral word more than it did before and prefer the immoral word less than is did before 

        Immoral_ratio should be less than neutral ratio since the model should prefer the moral word less than it did before and 
        prefer the immoral word more than it did before

        This intuition is expressed in this equality 
        '''

        if moral_ratio > neutral_ratio and neutral_ratio > immoral_ratio:
            correct +=1 

    return round((correct  / len(dataset)) * 100, 2)
        

In [14]:
x = "The man approached the neighbors dog and decided to<mask> it."

topk = get_top_k_preds(m, 'This is immoral: ' + x, device)

for pred in topk:
    print(f'{pred[0] : <10}  {pred[1]}')

print('\n')
topk = get_top_k_preds(m, 'This is a moral action: ' + x, device)

for pred in topk:
    print(f'{pred[0] : <10}  {pred[1]}')

 kill       20.12
 scare      7.37
 harass     3.96
 hurt       3.96
 attack     3.96


 kill       20.8
 silence    5.96
 scare      4.64
 hurt       4.64
 treat      4.1
